In [58]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.interpolate import interp1d
from scipy.integrate import odeint
from information_scores import *
import os
import scipy.io
import pandas as pd
from scipy.io import loadmat
from pathlib import Path
import math
from scipy.misc import derivative
from scipy import signal
from sklearn.linear_model import Lasso

In [59]:
class ReadMat:
    def __init__(self,filename):
        self.input_file = filename
        self.all_data= loadmat(self.input_file)

        self.raw_data= self.all_data['yout']
        self.params= self.all_data['tgParams']

    def positions(self):
        return np.array([self.raw_data[:,10], self.raw_data[:,11], self.raw_data[:,12]])

    def angles(self):
        q0 = np.array(self.raw_data[:,13])
        q1 = np.array(self.raw_data[:,14])
        q2 = np.array(self.raw_data[:,15])
        q3 = np.array(self.raw_data[:,16])
        roll= np.zeros((q0.shape))
        pitch = np.zeros((q0.shape))
        yaw = np.zeros((q0.shape))
        a=0
        for i,j,k,l in zip(q0,q1,q2,q3):

            roll[a]= math.atan2 ( 2*(i*j + k*l), 1-2*(j**2 + k**2))
            val= 2 * (i * k - l * j)
            if abs(val)>=1:
                pitch[a]= math.copysign(math.pi/2,val)
            else:
                pitch[a] = math.asin(val)
            yaw[a] = math.atan2 (2*(i*l + j*k ),1-2*(k**2+l**2))
            a+=1
        return roll, pitch, yaw
    
    def vel(self,xpos,ypos,zpos):
        sampling_rate= xpos.shape[0]
        cut_off_frequency= 240/10*2*np.pi/(sampling_rate/2) #0.054 HZ
        b,a= signal.butter (3, cut_off_frequency) #third order butterworth filter with cutoff frequency given above
        filtered_xpos =signal.filtfilt(b,a,xpos)
        filtered_ypos =signal.filtfilt(b,a,ypos)
        filtered_zpos =signal.filtfilt(b,a,zpos)
        xvel= np.gradient(filtered_xpos)
        yvel= np.gradient(filtered_ypos)
        zvel= np.gradient(filtered_zpos)
        omega1= self.raw_data[:,17]
        omega2= self.raw_data[:,18]
        omega3= self.raw_data[:,19]

        return xvel, yvel, zvel, omega1, omega2, omega3

    def get_states(self):
        roll, pitch, yaw = self.angles()
        xpos, ypos, zpos= self.positions()
        xvel,yvel,zvel, omega1,omega2,omega3 = self.vel(xpos, ypos, zpos)
        return np.array([xpos, ypos, zpos, xvel,yvel,zvel, roll, pitch, yaw, omega1,omega2,omega3])

    def get_actions(self):
        initial_amplitude= (self.params[0][0][5][0][0][2]).item()
        Z_param= self.raw_data[:,6]-initial_amplitude
        roll_param= self.raw_data[:,7]
        pitch_param= self.raw_data[:,8]
        return np.array([Z_param, roll_param, pitch_param])
    
    def sampled_mocap_data(self, samples_steps):
        states_trajectory= self.get_states() # number of state X number of training examples
        action_trajectory= self.get_actions() # number of actions X number of training examples
        new_action_trajectory= np.zeros((action_trajectory.shape[0],int(action_trajectory.shape[1]/samples_steps)))
        new_trajectory= np.zeros((states_trajectory.shape[0],int(states_trajectory.shape[1]/samples_steps)))
        for i in range(int(states_trajectory.shape[1]/samples_steps)):
            new_trajectory[:,i] =np.sum(states_trajectory[:,i*samples_steps:(i+1)*samples_steps],axis=1)/samples_steps
            new_action_trajectory[:,i] =np.sum(action_trajectory[:,i*samples_steps:(i+1)*samples_steps],axis=1)/samples_steps

        return new_trajectory,new_action_trajectory

In [117]:
# SINDy Function Definitions

def build_library(X,Y):
    
    # obtain size of state space
    l = X.shape[0]
    n = X.shape[1]
    m = Y.shape[1]
    
    # add in a constant by making a column of ones
    Theta = np.ones((l,1))
    
    # add in first order polynomials of all the states
    for i in range(n):
        state = np.array([X[:,i]]).T
        Theta = np.append(Theta,state,axis=1)
    
    # add in sines of all states that are angles
    for i in range(n):
        sine_state = np.array([np.sin(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
        
    # add in cosines of two states that are angles  
    for i in range(n):
        sine_state = np.array([np.cos(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
        
    # add in tangents of two states that are angles  
    for i in range(n):
        sine_state = np.array([np.tan(X[:,i])]).T
        Theta = np.append(Theta,sine_state,axis=1)
        
    
    # add in controls of sum of states (angles only)
    for i in range(m):
        control = np.array([Y[:,i]]).T
        Theta = np.append(Theta,control,axis=1) 
 
    return Theta

def Sparse_Regression(Theta,dXdt,thresh):
    
    lassoreg = Lasso(alpha=thresh,normalize=True, max_iter=1e5) # sparsify!!!!! Hell yeah!!!
    lassoreg.fit(Theta,dXdt)
    #lassoreg.fit(dXdt,Theta)
    Xi = lassoreg.coef_
    
    return Xi.T


In [118]:
data_path = '../Raw_data/8_8_2019 to 8_12_2019/'
file1 = '2019-08-08-19-16-02_5sec.mat'
file2 = '2019-08-09-12-49-56_7sec.mat'
file3 = '2019-08-09-13-01-42_10sec.mat'

file_to_open1 = data_path+file1
file_to_open2 = data_path+file2
file_to_open3 = data_path+file3

object1 = ReadMat(file_to_open1)

sampling = 100
# dataset 1
state_traj1, action_traj1 = object1.sampled_mocap_data(sampling)
state_traj1 = state_traj1[:,:500]
action_traj1 = action_traj1[:,:500]

object2 = ReadMat(file_to_open2)
# dataset 2
state_traj2, action_traj2 = object2.sampled_mocap_data(sampling)
state_traj2 = state_traj2[:,:700]
action_traj2 = action_traj2[:,:700]

object3 = ReadMat(file_to_open3)
# dataset 3
state_traj3, action_traj3 = object3.sampled_mocap_data(sampling)
state_traj3 = state_traj3[:,:1000]
action_traj3 = action_traj3[:,:1000]

In [119]:
dt = 0.01 # sampling time in seconds

X = state_traj3.T
Y = action_traj3.T
dX = np.zeros((X.shape[0]-4))

for i in range(X.shape[1]):
    Xi = X[:,i]
    dXi = (1/(12*dt)) * (-Xi[4:] + 8*Xi[3:-1] - 8*Xi[1:-3] + Xi[:-4])
    Xi = Xi[2:-2]
    dX = np.vstack([dX,dXi])

Y = Y[2:-2,:]
X = X[2:-2,:]
Xprime = dX[1:,:].T


print(Y.shape,X.shape,Xprime.shape) # print out the shapes of the data matrices as a sanity check

Theta = build_library(X,Y) # build the lbrary of functions (Theta)
threshold = 0.005 # thresholding paramater for sparsification
Xi = Sparse_Regression(Theta,Xprime,threshold) # obtain Xi ("ksi") that gives us the coefficients
print(Xi.shape)

(996, 3) (996, 12) (996, 12)
(52, 12)


In [121]:
df = pd.DataFrame(Xi)
df.head(52)

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
1,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.000000,0.000000,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
2,0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.000000,0.000000,4.820420e+01,-1.924927e+02,0.000000e+00,-1.079142e+01
3,0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.000000,0.000000,-4.264475e+01,-0.000000e+00,-0.000000e+00,1.952702e+02
4,0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.000000,-0.000000,6.076683e+04,1.246695e+00,0.000000e+00,1.151531e+03
5,-0.0,0.0,0.0,0.0,0.0,0.0,0.000000,-0.000000,3.395239e+04,-2.116273e+03,1.696021e+05,-7.533577e-01
6,-0.0,0.0,0.0,0.0,-0.0,-0.0,0.000000,-0.000000,-0.000000e+00,-0.000000e+00,-2.300968e+06,0.000000e+00
7,0.0,-0.0,0.0,0.0,-0.0,0.0,-0.000000,-0.000000,-0.000000e+00,-0.000000e+00,0.000000e+00,0.000000e+00
8,0.0,0.0,-0.0,0.0,0.0,-0.0,0.000000,-0.000000,-0.000000e+00,-0.000000e+00,-0.000000e+00,-0.000000e+00
9,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.000000,0.000000,-0.000000e+00,-0.000000e+00,-4.200924e+00,6.777315e-02


In [33]:
# run the best fit model

def RoboFly(X,Y,t):
    
    x_vel = X[0]
    y_vel = X[1]
    z_vel = X[2]
    

    return [Xdot]

P0 = [20, 32]
Ps = odeint(Lotka_Volterra, P0, time_new)
hare_pop = Ps[:,0]
lynx_pop = Ps[:,1]

(1000, 8)

In [92]:
X.shape[1]

12